### ライブラリのインポート

In [105]:
import pandas as pd
import re
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# アトラクション名の辞書
attractions = {
    0: 'ソアリン',
    1: 'トイストーリーマニア',
    2: 'タワーオブテラー',
    3: 'センターオブジアース',
    4: 'インディージョーンズ',
    5: 'レイジングスピリッツ',
    6: 'マジックランプシアター',
    7: 'タートルトーク'
}

### dfの読み込み・ソート・成形

In [106]:
# CSVファイルの読み込み
df = pd.read_csv('data/review/review.csv')
df = df.sort_values(by='attraction_id')

# レビュー列を抽出して1つのテキストに結合
df['all_reviews'] = df.apply(lambda row: ' '.join([str(row[col]) for col in df.columns[2:] if pd.notna(row[col])]), axis=1)
df_reviews = df[['attraction_id','attraction_name','all_reviews']]
df_reviews = df_reviews.head(8)
df_reviews['attraction_id'] = df_reviews['attraction_id'].astype(int)
display(df_reviews)

,attraction_id,attraction_name,all_reviews
1,0,ソアリン,ラインカットぎりぎりで並べたソアリン。\n\nネタバレしない程度にオススメします。\n世界で...
0,1,トイストーリーマニア,パークの営業が再開した7月最初の日曜日にインしてきました。\n久しぶりすぎるトイマニ体験\n...
2,2,タワーオブテラー,1万人の人数制限中のタワーオブテラー\n待ち時間\n 1万人の人数制限中のため、インディージ...
3,3,センターオブジアース,このアトラクションは絶叫嫌いの人にはキツいけど、好きな人には楽しい。\n最後、加速し落下。加...
4,4,インディージョーンズ,私は7月の平日にディズニーシーでこのアトラクションに乗りました!\n私がいつもディズニーシー...
5,5,レイジングスピリッツ,バッググラウンドに思いを馳せて\nもう何年も乗ってるレイジングスピリッツ。待ち時間に暇だから...
6,6,マジックランプシアター,面白い！\nアシーム君が可愛く、シャバーンのしゃべりが面白くってあっという間でした！\n\n...
7,7,タートルトーク,５月１２日よりドリーのキャラクターが登場したと言うことで行ってきました。\n\nネタバレあり...


### 文の特殊文字除去や小文字化、ストップワードの除去、形態素解析

In [107]:
# 正規表現を使って特殊文字を除去する関数
def clean_text(text):
    # 正規表現パターンで特殊文字を空白に置換
    text = re.sub(r'[^\w\s]', ' ', text)
    # 余分な空白を除去して返す
    return ' '.join(text.split())

# テキスト前処理（小文字化、ストップワードの除去、形態素解析）
def preprocess_text(text):
    # 小文字化
    text = text.lower()
    
    # 特殊文字の除去
    text = clean_text(text)
    
    # MeCabによる形態素解析
    tagger = MeCab.Tagger("-Owakati")  # 分かち書き形式で解析
    text = tagger.parse(text).strip()  # 不要な空白を除去
    
    return text

# レビューの前処理
df_reviews['clean_reviews'] = df_reviews['all_reviews'].apply(preprocess_text)

### TF-IDF　出現頻度の高い単語

In [108]:
# CountVectorizerを使用して各アトラクションのレビューから単語の出現頻度を計算
from sklearn.feature_extraction.text import CountVectorizer

def top_words_per_attraction(df, attraction_id, top_n=3):
    # 対象アトラクションのレビューを取得
    reviews = df[df['attraction_id'] == attraction_id]['all_reviews'].values
    
    # CountVectorizerを初期化
    vectorizer = CountVectorizer()
    
    # テキストをベクトル化して単語の出現回数を計算
    X = vectorizer.fit_transform(reviews)
    
    # 各単語の出現回数の合計を計算
    word_counts = X.sum(axis=0)
    
    # 単語とその出現回数を取得してDataFrameに格納
    word_counts_df = pd.DataFrame({
        'word': vectorizer.get_feature_names(),
        'count': word_counts.flat
    })
    
    # 出現回数の多い順にソートして上位n件を取得
    top_words = word_counts_df.sort_values(by='count', ascending=False).head(top_n)
    
    return top_words

In [109]:
# 各アトラクションごとに出現回数の多い単語を表示
for attraction_id in attractions.keys():
    attraction_name = attractions[attraction_id]
    print(f"--- {attraction_name} ---")
    top_words = top_words_per_attraction(df_reviews, attraction_id)
    print(top_words)
    print()

--- ソアリン ---
          word  count
447       ソアリン     11
519  ファンタスティック      9
523       フライト      7

--- トイストーリーマニア ---
    word  count
586   トイ     10
364   また      9
249   ただ      8

--- タワーオブテラー ---
    word  count
698   オブ     26
804  タワー     17
830  テラー     12

--- センターオブジアース ---
     word  count
218    オブ      7
256  センター      5
3      10      3

--- インディージョーンズ ---
      word  count
284   インディ      7
154     ただ      4
360  ジョーンズ      4

--- レイジングスピリッツ ---
          word  count
209      ジョーンズ      3
181  イクチュラコアトル      3
175  アクトゥリクトゥリ      3

--- マジックランプシアター ---
      word  count
181  シャバーン      4
226  ストーリー      3
77     そして      3

--- タートルトーク ---
         word  count
325     クラッシュ      6
467  ファインディング      4
418     スティッチ      3



/Users/INIAD/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/INIAD/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/INIAD/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/INIAD/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get

### 単語から予測されるアトラクションを表示する

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDFベクトル化
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_reviews['clean_reviews'])

def find_related_attraction(word):
    # 入力単語を前処理
    word = preprocess_text(word)
    
    # 入力単語をベクトル化
    word_vec = vectorizer.transform([word])
    
    # 類似度計算
    cosine_similarities = cosine_similarity(word_vec, tfidf_matrix).flatten()
    
    # 最も類似度が高いアトラクションのインデックスを取得
    related_index = cosine_similarities.argmax()
    
    # 類似度が低い場合や該当するアトラクションがない場合は8を返す
    if cosine_similarities[related_index] == 0:
        return '該当アトラクションなし'
    
    # 対応するアトラクション名を取得
    related_attraction = df_reviews.iloc[related_index]['attraction_id']
    
    return attractions[related_attraction]

In [111]:
# テスト
word = '筋肉痛'
print(find_related_attraction(word))

トイストーリーマニア
